### Описание проекта

Модель рекомендаций, построенная на данных для REKKO Challenge.
Переобучение и скоринг раз в 2 недели, последующая загрузка скоров на кластер.
При поступлении user_id из кафки - селектим по ключу из кассандры рекоммендации. Если для такого id нет рекомендаций - возвращаем 5 самых популярных фильмов. 

### Импорт библиотек

In [1]:
# ! pip install pandas numpy scipy implicit pprint tqdm

In [2]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from pprint import pprint
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import normalize
from scipy.sparse import spdiags
from scipy.sparse import vstack

import lightgbm as lgb

### Загрузка данных

In [3]:
data_path = './data'

`catalogue.json` содержит анонимизированную метаинформацию о доступных в сервисе фильмах и сериалах.

In [4]:
with open(os.path.join(data_path, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}

In [5]:
len(catalogue)

10200

In [6]:
pprint(catalogue[100])

{'attributes': [18441,
                16300,
                16580,
                18770,
                18771,
                18643,
                396,
                18772,
                3771,
                18773,
                910,
                18774,
                16364,
                3277],
 'availability': ['purchase', 'rent'],
 'duration': 80,
 'feature_1': 6064738.740195342,
 'feature_2': 0.752750538,
 'feature_3': 4,
 'feature_4': 0.9537104605,
 'feature_5': 0.0,
 'type': 'movie'}


 - `attributes` — мешок атрибутов
 - `availability` — доступность (может содержать значения `purchase`, `rent` и `subscription`)
 - `duration` — длительность в минутах, округлённая до десятков (продолжительность серии для сериалов и многосерийных фильмов)
 - `feature_1..5` — пять анонимизированных вещественных и порядковых признаков
 - `type` — принимает значения `movie`, `multipart_movie` или `series`

---

`test_users.json` содержит список пользователей, для которых необходимо построить предсказание

In [7]:
with open(os.path.join(data_path, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

---

`transactions.csv` — список всех транзакций за определённый период времени

In [8]:
%%time
transactions = pd.read_csv(
    os.path.join(data_path, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 2.92 s, sys: 274 ms, total: 3.2 s
Wall time: 3.21 s


In [9]:
transactions.head(3)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50


 - `element_uid` — идентификатор элемента
 - `user_uid` — идентификатор пользователя
 - `consumption_mode` — тип потребления (`P` — покупка, `R` — аренда, `S` — просмотр по подписке)
 - `ts` — время совершения транзакции или начала просмотра в случае просмотра по подписке
 - `watched_time` — число просмотренных по транзакции секунд
 - `device_type` — анонимизированный тип устройства, с которого была совершена транзакция или начат просмотр
 - `device_manufacturer` — анонимизированный производитель устройства, с которого была совершена транзакция или начат просмотр

---

`ratings.csv` содержит информацию о поставленных пользователями оценках

In [10]:
%%time
ratings = pd.read_csv(
    os.path.join(data_path, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)

CPU times: user 95.8 ms, sys: 22.7 ms, total: 119 ms
Wall time: 119 ms


In [11]:
ratings.head(3)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07


 - `rating` — поставленный пользователем рейтинг (от `0` до `10`)

---

`bookmarks.csv` содержит информацию об элементах, добавленных пользователями в список «Избранное»

In [12]:
%%time
bookmarks = pd.read_csv(
    os.path.join(data_path, 'bookmarks.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64
    }
)

CPU times: user 171 ms, sys: 24.9 ms, total: 196 ms
Wall time: 197 ms


In [13]:
bookmarks.head(3)

,user_uid,element_uid,ts
0,301135,7185,4.430516e+07
1,301135,4083,4.430516e+07
2,301135,10158,4.430516e+07


### Подготовка и тестирование модели

In [14]:
test_size_ts = 604800

data_train = transactions[transactions['ts'] < transactions['ts'].max() - test_size_ts]
data_test = transactions[transactions['ts'] >= transactions['ts'].max() - test_size_ts]

data_train.head(2)

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
2508834,4363,293613,S,4.370038e+07,65,0,11
2508835,7449,592643,S,4.370038e+07,941,3,99


#### Подготовим двухуровневую модель

In [15]:
# Новым пользователям будем рекомендовать топ-5
popularity_top = data_train['element_uid'].value_counts().keys().tolist()[:5]
popularity_top

[2714, 747, 3916, 2245, 6127]

In [16]:
# Подготовка матрицы item-user

# берем пользователей, просмотревших не менее 2 фильмов


matrix_users = data_train['user_uid'].value_counts().loc[lambda x : x > 2].keys().tolist()
user_to_col = {}


for col_id, user_id in enumerate(matrix_users):
    user_to_col[user_id] = col_id


# строим индекс obj_id -> row_id, где row_id - идентификатор строки в матрице
# берем только фильмы, которые просмотрело не менее 10 пользователей


matrix_items = data_train['element_uid'].value_counts().loc[lambda x : x > 10].keys().tolist()

obj_to_row = {}

for row_id, obj_id in enumerate(matrix_items):
    obj_to_row[obj_id] = row_id
    
print (f"Количество пользователей: {len(user_to_col)}")
print (f"Количество объектов: {len(obj_to_row)}")

Количество пользователей: 404008
Количество объектов: 7088


In [17]:
# Подготовим таблицу для замера результата
result = data_test.groupby('user_uid')['element_uid'].unique().reset_index()
result.columns=['user_uid', 'actual']

result = result.loc[result['user_uid'].isin(matrix_users)] 

result.head(2)

,user_uid,actual
0,0,"[8739, 603, 3353, 6874, 51, 3623, 8864, 7619, ..."
1,1,"[9932, 9412, 7642, 1844, 5665]"


In [18]:
%%time

# создаем матрицу нужных размеров

matrix = lil_matrix(((len(user_to_col), (len(obj_to_row)))))  

# заполняем матрицу

for index, row in data_train.iterrows():
    row_id = obj_to_row.get(row['element_uid'])
    col_id = user_to_col.get(row['user_uid'])
    if row_id is not None and col_id is not None:
        matrix[col_id, row_id] = 1
        
for index, row in ratings.iterrows():
    row_id = obj_to_row.get(row['element_uid'])
    col_id = user_to_col.get(row['user_uid'])
    if row_id is not None and col_id is not None:
        matrix[col_id, row_id] *= row['rating']    
        
for index, row in bookmarks.iterrows():
    row_id = obj_to_row.get(row['element_uid'])
    col_id = user_to_col.get(row['user_uid'])
    if row_id is not None and col_id is not None:
        matrix[col_id, row_id] *= 2
        
percent = float(matrix.nnz) / len(obj_to_row) / len(user_to_col) * 100
print(f"Процент заполненности матрицы: {percent}")

Процент заполненности матрицы: 0.24733050958068328
CPU times: user 10min 53s, sys: 1.09 s, total: 10min 54s
Wall time: 10min 55s


In [19]:
# нормализуем исходную матрицу 
normalized_matrix = normalize(matrix.tocsr()).tocsr()

In [20]:
model = AlternatingLeastSquares(factors=15, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(normalized_matrix).T.tocsr(), show_progress=True)

In [21]:
def get_recommendations(user, model, N=5):
    res = [obj_to_row.get(rec[0]) for rec in 
                    model.recommend(userid=user_to_col.get(user), 
                                    user_items=csr_matrix(normalized_matrix).tocsr(),   
                                    N=N, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)]   
    return res

In [22]:
%%time
    
result['als'] = result['user_uid'].apply(lambda x: get_recommendations(x, model=model, N=15))

CPU times: user 7min 25s, sys: 51.8 s, total: 8min 16s
Wall time: 2min 4s


In [23]:
# Подготовим второй уровень уровень - LGBboost: добавим признаки для обучения и построим модель 

users_lvl_2 = pd.DataFrame(data_train['user_uid'].unique())
users_lvl_2.columns = ['user_uid']
users_lvl_2 = users_lvl_2.loc[users_lvl_2['user_uid'].isin(matrix_users)] 

users_lvl_2['candidates'] = users_lvl_2['user_uid'].apply(lambda x: get_recommendations(x, model=model, N=30))

In [24]:
users_lvl_2_val = pd.DataFrame(data_test['user_uid'].unique())
users_lvl_2_val.columns = ['user_uid']

users_lvl_2_val = users_lvl_2_val[users_lvl_2_val['user_uid'].isin(matrix_users)]

users_lvl_2_val['candidates'] = users_lvl_2_val['user_uid'].apply(lambda x: get_recommendations(x, model=model, N=30))

In [25]:
df = pd.DataFrame({'user_uid':users_lvl_2.user_uid.values.repeat(len(users_lvl_2.candidates[0])),
                 'element_uid':np.concatenate(users_lvl_2.candidates.values)})

In [26]:
df_val = pd.DataFrame({'user_uid':users_lvl_2_val.user_uid.values.repeat(len(users_lvl_2_val.candidates[0])),
                 'element_uid':np.concatenate(users_lvl_2_val.candidates.values)})

In [27]:
targets_lvl_2 = data_train[['user_uid', 'element_uid']].copy()
targets_lvl_2['target'] = 1   

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_uid', 'element_uid'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [28]:
targets_lvl_2_val = data_test[['user_uid', 'element_uid']].copy()
targets_lvl_2_val['target'] = 1   

targets_lvl_2_val = df_val.merge(targets_lvl_2_val, on=['user_uid', 'element_uid'], how='left')

targets_lvl_2_val['target'].fillna(0, inplace= True)

In [29]:
def get_feats(element, i):
    
    try:
        feat = catalogue[element][f'feature_{i}']
    except:
        feat = 0
     
    return feat

In [30]:
def get_user_feats(user, i):
    
    try:
        feat = model.user_factors[user_to_col.get(user)][i]
    except:
        feat = 0
     
    return feat

In [31]:
def get_duration(element):
    
    try:
        feat = catalogue[element]['duration']
    except:
        feat = 0
     
    return feat

In [32]:
for i in range(1, 6):
    targets_lvl_2[f'feature_{i}'] = targets_lvl_2['element_uid'].apply(lambda x: get_feats(x, i))
    targets_lvl_2_val[f'feature_{i}'] = targets_lvl_2_val['element_uid'].apply(lambda x: get_feats(x, i))

In [33]:
for i in range(15):
    targets_lvl_2[f'user_feature_{i}'] = targets_lvl_2['user_uid'].apply(lambda x: get_duration(x))
    targets_lvl_2_val[f'user_feature_{i}'] = targets_lvl_2_val['user_uid'].apply(lambda x: get_duration(x)) 

In [34]:
targets_lvl_2['duration'] = targets_lvl_2['element_uid'].apply(lambda x: get_feats(x, i))
targets_lvl_2_val['duration'] = targets_lvl_2_val['element_uid'].apply(lambda x: get_feats(x, i))

In [35]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

X_test = targets_lvl_2_val.drop('target', axis=1)
y_test = targets_lvl_2_val[['target']]

In [36]:
X_train['element_uid'] = pd.to_numeric(X_train['element_uid'], downcast='float', errors='coerce')
X_test['element_uid'] = pd.to_numeric(X_test['element_uid'], downcast='float', errors='coerce')

In [37]:
model_lgb = lgb.LGBMClassifier(objective='binary', max_depth=7)

In [38]:
model_lgb.fit(
    X=X_train, y=y_train)

y_score = model_lgb.predict(X_test)

/Users/ekaterinarazdumina/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [39]:
X_test['result'] = y_score

result_val_2 = X_test[['user_uid', 'element_uid', 'result']]
result_val_2 = result_val_2.loc[result_val_2['result'] == 1]
result_val_2 = result_val_2.groupby('user_uid')['element_uid'].apply(list).to_frame()

In [40]:
result = result.merge(result_val_2, on='user_uid', how='left')
result.rename(columns={'element_uid': 'lvl2_raw'}, inplace=True)

#### Замерим качество

In [41]:
def precision_at_k(recommended_list, watched_list, k=5):
    
    watched_list = np.array(watched_list)
    
    try:
        recommended_list = np.array([x for x in recommended_list if x != None])    
        recommended_list = recommended_list[:k]
    except:
        recommended_list = np.array(popularity_top)
    
    flags = np.isin(watched_list, recommended_list)
    precision = flags.sum() / k
    
    
    return precision

In [42]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.0008273712856076899

In [43]:
result.apply(lambda row: precision_at_k(row['lvl2_raw'], row['actual']), axis=1).mean()

0.01728042838958597

In [44]:
result

,user_uid,actual,als,lvl2_raw
0,0,"[8739, 603, 3353, 6874, 51, 3623, 8864, 7619, ...","[None, 6499, 6491, 159, 234, 1881, None, None,...",NaN
1,1,"[9932, 9412, 7642, 1844, 5665]","[35, 5248, 4301, None, 6491, 6499, None, 1984,...",NaN
2,2,[9206],"[3480, 5278, None, 734, 4644, 1529, 3794, 6886...",NaN
3,3,"[2052, 5057, 7513, 6440, 5947, 8647, 8101, 210...","[None, 1977, 1235, 6499, 444, None, 5805, 2841...",NaN
4,5,"[1911, 3378, 1565]","[3062, None, None, 4311, None, None, 5151, Non...",NaN
...,...,...,...,...
182259,593473,"[6103, 3359, 2760, 82, 4926, 7580]","[None, None, 6951, 601, None, 7087, None, None...",NaN
182260,593478,[1521],"[None, None, None, None, None, None, 3062, 708...",NaN
182261,593482,"[9179, 2383, 8420]","[6491, None, 3480, 6499, None, 6886, 3833, 198...",NaN
182262,593484,[9932],"[None, 1984, 1611, 4071, 5357, None, 4341, Non...",NaN
